In [57]:
from river import stream,tree,metrics
import utils
from encoding import prefix_bin
import csv
import copy
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import pickle as pkl
import matplotlib.pyplot as plt
import os,json
import datetime
from collections import deque
from sklearn.tree import DecisionTreeClassifier


In [2]:
def performance_measurement(metrics, type='Accuracy'):
    if type == 'Accuracy':
        return_value = metrics.Accuracy()
    elif type =='F1':
        return_value = metrics.F1()
    elif type =='ROCAUC':
        return_value = metrics.ROCAUC()
    elif type == 'WeightedF1':
        return_value = metrics.WeightedF1()
        
    return return_value

In [3]:
def display_progress(row_counting, total_length, interval=2000):
    if rowcounter%interval == 0:
        print(round(rowcounter*100/totallength,2) ,'%', 'Case finished: %s'%(casecount), 'Running cases: %s'%(len(streaming_db)))

In [4]:
def update_test_cases(new_case, test_cases):
    test_cases.append(new_case)
    if len(test_cases) > test_size:
        test_cases.popleft()

In [9]:
# Experiment settings
'./DATA/logs/synthetic_log_bc1.csv'
dataset_label = 'synthetic_log_bc1'

# Invoke parameters for dataset
window_size = 100
test_size = 30
with open('./dataset_parameters.json','r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']

for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    dataset_loc = './DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('./result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'hatc'
    if classifier == 'htc':
        streaming_classifier = tree.HoeffdingTreeClassifier(grace_period=10, split_criterion='info_gain')
    elif classifier =='hatc':
        streaming_classifier = tree.HoeffdingAdaptiveTreeClassifier(grace_period=10, split_criterion='info_gain' )
    elif classifier == 'efdt':
        streaming_classifier = tree.ExtremelyFastDecisionTreeClassifier(grace_period=10, split_criterion='info_gain')
        
    dataset = stream.iter_csv(
            dataset_loc
            )

    totallength = len(list(dataset))

    dataset = stream.iter_csv(
            dataset_loc,
            # drop=['Complete Timestamp'],
            target='outcome'
            )
    enctype = 'Index-base'
    
    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    save_model = {}
    casecount = 0
    rowcounter = 0
    finished_db ={}
    running_case = 0
    window_acc_dict = {}
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    usedingrace = set()
    
    
    '''
    Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
    and obtain feature matrix to transform future events
    '''
    for x,y in dataset:
#         display_progress(rowcounter, totallength)
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length        
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            usedingrace.add(caseid)
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            if casecount < 200:
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                        feature_matrix['prefix_%s'%(maximum_prefix)]=set()
                        training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier,
                                                                   0]
                    feature_list = list(streaming_db[caseid][maximum_prefix-1].encoded.keys())
                    for x in feature_list: feature_matrix['prefix_%s'%(maximum_prefix)].add(x) 

                update_test_cases(streaming_db[caseid], test_cases)
            else:
                break
                
    for cid in list(usedingrace):
        case_length = len(streaming_db[cid])
        if case_length >= maximum_prefix:
            x = streaming_db[cid][maximum_prefix-1]
            if x.prefix_length != 0:
                x.encoded = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                outcome = x.true_label
                training_models['prefix_%s'%(x.prefix_length)][0].learn_one(x.encoded,outcome)
                training_models['prefix_%s'%(x.prefix_length)][1] +=1

    model_update_count = training_models['prefix_%s'%(maximum_prefix)][1]
    prediction_result[maximum_prefix][model_update_count] = {}
    y_truelist = []
    y_predlist = []
    for case in test_cases:
        if len(case) >= maximum_prefix:
            x = case[maximum_prefix-1]
            if x.prefix_length !=0:
                model = training_models['prefix_%s'%(x.prefix_length)][0]
                y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba_one(x.encoded)
                y_truelist.append(x.true_label)
                y_predlist.append(y_pred)

    prediction_result[maximum_prefix][model_update_count]['y_true'] = y_truelist
    prediction_result[maximum_prefix][model_update_count]['y_pred'] = y_predlist
    
    '''
    Streaming event label prediction start.
    - Test and training steps are executed when case finished/ event arrived with label
    '''
    for i in streaming_db.keys(): usedingrace.add(i)
    streaming_db ={}
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0
    for x,y in dataset:
#         display_progress(rowcounter, totallength)

        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        # Check label possible
        # x = utils.set_label(x)
        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
        x.pop('caseid')
        x.pop('outcome')

        if caseid not in usedingrace:
            case_bin = prefix_bin(caseid, x)
            case_bin.set_enctype(enctype)

            if caseid not in list(streaming_db.keys()):
                case_bin.set_prefix_length(1)    
                streaming_db[caseid] = []
                running_case +=1
            elif caseid in finished_caseid:
                pass
            else:
                case_bin.set_prefix_length(len(streaming_db[caseid])+1)
                case_bin.set_prev_enc(streaming_db[caseid][-1])

            # Encode event and cases and add to DB
            case_bin.update_truelabel(outcome)   
            case_bin.update_encoded(catattrs=catatars,enctype=enctype)
            ts = case_bin.event['ts']

            if case_bin.prefix_length == maximum_prefix:
                case_bin.encoded = utils.readjustment_training(case_bin.encoded, feature_matrix['prefix_%s'%(case_bin.prefix_length)])
            streaming_db[caseid].append(case_bin)

            # Detect label appeared case 
            if outcome != '' and caseid not in finished_caseid:
                finished_caseid.add(caseid)

                # Adding newly finished case to training set.
                casecount +=1    

                # Modify encoded attributes of cases with feature matrix
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:

                    update_test_cases(streaming_db[caseid], test_cases)
                    streaming_db[caseid][maximum_prefix-1].update_truelabel(outcome)
                    x = streaming_db[caseid][maximum_prefix-1].encoded
                    prefix_length =streaming_db[caseid][maximum_prefix-1].prefix_length
                    training_models['prefix_%s'%(prefix_length)][0].learn_one(x, outcome)
                    training_models['prefix_%s'%(prefix_length)][1] +=1

                    y_pred = training_models['prefix_%s'%(prefix_length)][0].predict_one(x)
                    model_update_count = training_models['prefix_%s'%(prefix_length)][1]
                    save_model['%s_%s'%(prefix_length, model_update_count)] = training_models['prefix_%s'%(streaming_db[caseid][maximum_prefix-1].prefix_length)][0]

                    y_truelist = []
                    y_predlist = []

                    for case in test_cases:
                        if len(case) >= maximum_prefix:
                            x = case[maximum_prefix-1]
                            length = x.prefix_length
                            if length != 0:
                                y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba_one(x.encoded)
                                y_truelist.append(x.true_label)
                                y_predlist.append(y_pred)

                    prediction_result[maximum_prefix][model_update_count] = {}
                    prediction_result[maximum_prefix][model_update_count]['y_true'] = y_truelist
                    prediction_result[maximum_prefix][model_update_count]['y_pred'] = y_predlist
                    if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
                        cdhappend[maximum_prefix] = model_update_count

    for evaluation_method in ['Accuracy', 'F1', 'ROCAUC']:
        prefix = maximum_prefix
        update_count =[]
        evaluation_list = []
        length_list = []
        for update in prediction_result[prefix].keys():
            y_true = prediction_result[prefix][update]['y_true']
            y_pred_prob = prediction_result[prefix][update]['y_pred']
            y_pred = [max(x, key=x.get) for x in y_pred_prob]
            update_count.append(update)

            if evaluation_method =='Accuracy':
                value = accuracy_score(y_true, y_pred)
            elif evaluation_method =='F1':
                value = f1_score(y_true, y_pred, average='macro')
        #             warnings.filterwarnings("ignore")

            elif evaluation_method =='ROCAUC':
                y_pred_prob_ =[]
                for x in y_pred_prob:
                    if 'False' in x.keys():
                        x_false = x['False']
                    else:
                        x_false = 0
                    if 'True' in x.keys():
                        x_true = x['True']
                    else:
                        x_true = 0
                    y_pred_prob_.append([x_false, x_true])
                y_pred_prob = np.array(y_pred_prob_)
                try:
                    value = roc_auc_score(y_true, y_pred_prob[:,1])
                except:
                    value = 0
            evaluation_list.append(value)
        plt.figure(figsize=(15,8))
        plt.plot(update_count, evaluation_list, label = 'Accuracy')
        #     plt.plot(update_count, length_list)
        if dataset_label == 'bpic15' or dataset_label == 'bpic17':
            pass
        else:
            plt.axvline(cdhappend[prefix], color = 'black', linestyle ='--', label = 'Concept drift')
        plt.title(prefix)
        plt.xlabel('Model update count')
        plt.legend(bbox_to_anchor=(0.65, -0.1), ncol=2)
        plt.ylim(0,1)
    #     plt.show()

        if os.path.exists('./img/%s/%s/%s'%(dataset_label, classifier, 'Model updates')) == False:
            os.makedirs('./img/%s/%s/%s'%(dataset_label, classifier, 'Model updates'))
        if os.path.exists('./result/%s/%s/%s'%(dataset_label, classifier, 'Model updates')) == False:
            os.makedirs('./result/%s/%s/%s'%(dataset_label, classifier, 'Model updates'))


        with open('./result/%s/%s/Model updates/prefix_%s_%s update.pkl'%(dataset_label, classifier, prefix, evaluation_method), 'wb') as f:
            pkl.dump(prediction_result, f)
        plt.savefig('./img/%s/%s/Model updates/prefix_%s_%s update.png'%(dataset_label, classifier, prefix, evaluation_method), dpi=250)
        plt.cla()
        plt.clf()
        plt.close()

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}
1
2
3
4
5
6
7
8
9
10
11
12


In [58]:
# Experiment settings
'./DATA/logs/synthetic_log_bc1.csv'
dataset_label = 'synthetic_log_bc1'

# Invoke parameters for dataset
window_size = 100
test_size = 30
with open('./dataset_parameters.json','r') as json_file:
    parameters = json.load(json_file)[dataset_label]
    print(parameters)
    key_pair = parameters['key_pair']
    catatars = parameters['categorical_attrs']
    maximum_prefixs = parameters['maximum_prefix']

for maximum_prefix in range(1, maximum_prefixs+1):
    print(maximum_prefix)
    dataset_loc = './DATA/logs/'+ dataset_label +'.csv'
    try:
        os.makedirs('./result/%s'%(dataset_label))
    except:
        pass

    # Set streaming classifier
    classifier = 'hatc'
    if classifier == 'htc':
        streaming_classifier = tree.HoeffdingTreeClassifier(grace_period=10, split_criterion='info_gain')
    elif classifier =='hatc':
        streaming_classifier = tree.HoeffdingAdaptiveTreeClassifier(grace_period=10, split_criterion='info_gain' )
    elif classifier == 'efdt':
        streaming_classifier = tree.ExtremelyFastDecisionTreeClassifier(grace_period=10, split_criterion='info_gain')
        
    dataset = stream.iter_csv(
            dataset_loc
            )

    totallength = len(list(dataset))

    dataset = stream.iter_csv(
            dataset_loc,
            # drop=['Complete Timestamp'],
            target='outcome'
            )
    enctype = 'Index-base'
    
    streaming_db ={}
    training_models ={}
    test_cases = deque()
    feature_matrix ={}

    save_model = {}
    casecount = 0
    rowcounter = 0
    finished_db ={}
    running_case = 0
    window_acc_dict = {}
    prediction_result = dict()
    for i in range(1, maximum_prefix+1): prediction_result[i] = {}
    finished_caseid = set()
    usedingrace = set()
    
    
    '''
    Before test and training streaming event predictive monitoring, grace period is preceded to initialize model by prefix length
    and obtain feature matrix to transform future events
    '''
    for x,y in dataset:
#         display_progress(rowcounter, totallength)
        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        x['outcome'] =y 
        # Initialize case by prefix length        
        caseid = x['caseid']
        outcome = x['outcome']
    #     progress = x['progress']

        x.pop('caseid')
        x.pop('outcome')

    #     x.pop('progress')

        case_bin = prefix_bin(caseid, x)
        case_bin.set_enctype(enctype)

        if caseid not in list(streaming_db.keys()):
            case_bin.set_prefix_length(1)    
            streaming_db[caseid] = []
        elif caseid in finished_caseid:
            pass
        else:
            case_bin.set_prefix_length(len(streaming_db[caseid])+1)
            case_bin.set_prev_enc(streaming_db[caseid][-1])

        # Encode event and cases and add to DB
        case_bin.update_truelabel(outcome)   
        case_bin.update_encoded(catattrs=catatars,enctype=enctype)
        ts = case_bin.event['ts']
        streaming_db[caseid].append(case_bin)
        # Detect label appeared case 
        if outcome != '' and caseid not in finished_caseid:
            usedingrace.add(caseid)
            for i in streaming_db[caseid]:
                i.update_truelabel(outcome)
            finished_caseid.add(caseid)
            # Adding newly finished case to training set.    
            casecount +=1
            # Grace period to collect feature matrix
            if casecount < 200:
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:
                    if 'prefix_%s'%(maximum_prefix) not in list(feature_matrix.keys()):
                        feature_matrix['prefix_%s'%(maximum_prefix)]=set()
                        training_models['prefix_%s'%(maximum_prefix)] = [streaming_classifier,
                                                                   0]
                    feature_list = list(streaming_db[caseid][maximum_prefix-1].encoded.keys())
                    for x in feature_list: feature_matrix['prefix_%s'%(maximum_prefix)].add(x) 

            else:
                break
    casecount_=0
    for caseid in list(usedingrace):

        case_length = len(streaming_db[caseid])
        if case_length >= maximum_prefix:
            x = streaming_db[caseid][maximum_prefix-1]
            if x.prefix_length != 0:            
                x.encoded = utils.readjustment_training(x.encoded, feature_matrix['prefix_%s'%(maximum_prefix)])
                outcome = x.true_label
                training_models['prefix_%s'%(x.prefix_length)][0].learn_one(x.encoded,outcome)
                training_models['prefix_%s'%(x.prefix_length)][1] +=1

            update_test_cases(streaming_db[caseid], test_cases)

        prediction_result[maximum_prefix][casecount_] = {}
        y_truelist = []
        y_predlist = []
        for case in test_cases:
            if len(case) >= maximum_prefix:
                x = case[maximum_prefix-1]
                if x.prefix_length != 0:            
                    model = training_models['prefix_%s'%(x.prefix_length)][0]
                    y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba_one(x.encoded)
                    y_truelist.append(x.true_label)
                    y_predlist.append(y_pred)
        prediction_result[maximum_prefix][casecount_]['y_true'] = y_truelist
        prediction_result[maximum_prefix][casecount_]['y_pred'] = y_predlist
        casecount_ +=1
    
    '''
    Streaming event label prediction start.
    - Test and training steps are executed when case finished/ event arrived with label
    '''
    for i in streaming_db.keys(): usedingrace.add(i)
    streaming_db ={}
    cdhappend ={}
    for i in range(1, maximum_prefix+1): cdhappend[i] = 0
    for x,y in dataset:
#         display_progress(rowcounter, totallength)

        rowcounter +=1
        # Event stream change dictionary keys
        x = utils.dictkey_chg(x, key_pair)

        if dataset_label !='bpic15':
            x['ts'] = x['ts'][:-4]

        # Check label possible
        # x = utils.set_label(x)
        x['outcome'] =y 
        # Initialize case by prefix length
        caseid = x['caseid']
        outcome = x['outcome']
        x.pop('caseid')
        x.pop('outcome')

        if caseid not in usedingrace:
            case_bin = prefix_bin(caseid, x)
            case_bin.set_enctype(enctype)

            if caseid not in list(streaming_db.keys()):
                case_bin.set_prefix_length(1)    
                streaming_db[caseid] = []
                running_case +=1
            elif caseid in finished_caseid:
                pass
            else:
                case_bin.set_prefix_length(len(streaming_db[caseid])+1)
                case_bin.set_prev_enc(streaming_db[caseid][-1])

            # Encode event and cases and add to DB
            case_bin.update_truelabel(outcome)   
            case_bin.update_encoded(catattrs=catatars,enctype=enctype)
            ts = case_bin.event['ts']

            if case_bin.prefix_length == maximum_prefix:
                case_bin.encoded = utils.readjustment_training(case_bin.encoded, feature_matrix['prefix_%s'%(case_bin.prefix_length)])
            streaming_db[caseid].append(case_bin)

            # Detect label appeared case 
            if outcome != '' and caseid not in finished_caseid:
                finished_caseid.add(caseid)

                # Adding newly finished case to training set.
                casecount +=1    

                # Modify encoded attributes of cases with feature matrix
                case_length = len(streaming_db[caseid])
                if case_length >= maximum_prefix:

                    update_test_cases(streaming_db[caseid], test_cases)
                    streaming_db[caseid][maximum_prefix-1].update_truelabel(outcome)
                    x = streaming_db[caseid][maximum_prefix-1].encoded
                    prefix_length =streaming_db[caseid][maximum_prefix-1].prefix_length
                    training_models['prefix_%s'%(prefix_length)][0].learn_one(x, outcome)
                    training_models['prefix_%s'%(prefix_length)][1] +=1

                    y_pred = training_models['prefix_%s'%(prefix_length)][0].predict_one(x)
                    model_update_count = training_models['prefix_%s'%(prefix_length)][1]
                    
                    save_model['%s_%s'%(prefix_length, model_update_count)] = training_models['prefix_%s'%(streaming_db[caseid][maximum_prefix-1].prefix_length)][0]

                y_truelist = []
                y_predlist = []

                for case in test_cases:
                    if len(case) >= maximum_prefix:
                        x = case[maximum_prefix-1]
                        if x.prefix_length != 0:
                            length = x.prefix_length
                            y_pred = training_models['prefix_%s'%(x.prefix_length)][0].predict_proba_one(x.encoded)
                            y_truelist.append(x.true_label)
                            y_predlist.append(y_pred)

                prediction_result[maximum_prefix][casecount] = {}
                prediction_result[maximum_prefix][casecount]['y_true'] = y_truelist
                prediction_result[maximum_prefix][casecount]['y_pred'] = y_predlist
                if 'b1' not in caseid and cdhappend[maximum_prefix] == 0:
                    cdhappend[maximum_prefix] = model_update_count

    for evaluation_method in ['F1']:
        prefix = maximum_prefix
        update_count =[]
        evaluation_list = []
        length_list = []
        for update in prediction_result[prefix].keys():
            y_true = prediction_result[prefix][update]['y_true']
            y_pred_prob = prediction_result[prefix][update]['y_pred']
            y_pred_n = []
            for t in y_pred_prob:
                s = sorted(t.items(), key=lambda x: x[1], reverse=True)[0][0]
                y_pred_n.append(s)
            y_pred = y_pred_n
            update_count.append(update)

            if evaluation_method =='Accuracy':
                value = accuracy_score(y_true, y_pred)
            elif evaluation_method =='F1':
                value = f1_score(y_true, y_pred, average='macro')
        #             warnings.filterwarnings("ignore")

            elif evaluation_method =='ROCAUC':
                y_pred_prob_ =[]
                for x in y_pred_prob:
                    if 'False' in x.keys():
                        x_false = x['False']
                    else:
                        x_false = 0
                    if 'True' in x.keys():
                        x_true = x['True']
                    else:
                        x_true = 0
                    y_pred_prob_.append([x_false, x_true])
                y_pred_prob = np.array(y_pred_prob_)
                try:
                    value = roc_auc_score(y_true, y_pred_prob[:,1])
                except:
                    value = 0
            evaluation_list.append(value)
        plt.figure(figsize=(15,8))
        plt.plot(update_count, evaluation_list, label = 'Accuracy')
        #     plt.plot(update_count, length_list)
        if dataset_label == 'bpic15' or dataset_label == 'bpic17':
            pass
        else:
            plt.axvline(cdhappend[prefix], color = 'black', linestyle ='--', label = 'Concept drift')

        plt.title(prefix)
        plt.xlabel('Model update count')
        plt.legend(bbox_to_anchor=(0.65, -0.1), ncol=2)
        plt.ylim(0,1)
    #     plt.show()

        if os.path.exists('./img/%s/%s/%s'%(dataset_label, classifier, 'Finished cases')) == False:
            os.makedirs('./img/%s/%s/%s'%(dataset_label, classifier, 'Finished cases'))
        if os.path.exists('./result/%s/%s/%s'%(dataset_label, classifier, 'Finished cases')) == False:
            os.makedirs('./result/%s/%s/%s'%(dataset_label, classifier, 'Finished cases'))


        with open('./result/%s/%s/Finished cases/prefix_%s_%s update.pkl'%(dataset_label, classifier, prefix, evaluation_method), 'wb') as f:
            pkl.dump(prediction_result, f)
        plt.savefig('./img/%s/%s/Finished cases/prefix_%s_%s update.png'%(dataset_label, classifier, prefix, evaluation_method), dpi=250)
        plt.cla()
        plt.clf()
        plt.close()

{'key_pair': {'Case ID': 'caseid', 'Activity': 'activity', 'Complete Timestamp': 'ts'}, 'categorical_attrs': ['activity'], 'maximum_prefix': 12}
1
2
3
4
5
6
7
8
9
10
11
12


In [61]:
f1_score_list = []
for i in prediction_result[12]:
    y_true = prediction_result[12][i]['y_true']
    y_pred = prediction_result[12][i]['y_pred']
    
    y_pred_n = []
    for t in y_pred:
        
        s = sorted(t.items(), key=lambda x: x[1], reverse=True)[0][0]
#         print(s)
        y_pred_n.append(s)
    
#     print(y_true, y_pred_n)
    f1_score_list.append(f1_score(y_true = y_true, y_pred=y_pred_n, average='macro'))

In [63]:
print(np.mean(f1_score_list))

0.5619791408207828


In [11]:
test_cases[1][0].caseid

'IRO1_435'

In [7]:
training_models['prefix_12']

[HoeffdingAdaptiveTreeClassifier (
   grace_period=10
   max_depth=inf
   split_criterion="info_gain"
   split_confidence=1e-07
   tie_threshold=0.05
   leaf_prediction="nba"
   nb_threshold=0
   nominal_attributes=None
   splitter=GaussianSplitter (
     n_splits=10
   )
   bootstrap_sampling=True
   drift_window_threshold=300
   adwin_confidence=0.002
   binary_split=False
   max_size=100
   memory_estimate_period=1000000
   stop_mem_management=False
   remove_poor_attrs=False
   merit_preprune=True
   seed=None
 ),
 493]